In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

First, we read in the datasets with conditional probabilities of demographic groups and the dataset with raw counts.

In [2]:
data = pd.read_csv('../data/probabilities.csv')
raws = pd.read_csv(('../data/dataset.csv'))

Next, we add in some missing features from the raw counts dataset to the probabilities dataset. Specifically, we add `median_household_income` and `per_capita_income`, and we also throw in `per_capita_year` (computed by dividing `land_area_sqkm` by `persons_total` in the raws dataset).

In [3]:
temp = raws[['year','gisjoin','median_household_income', 'per_capita_income']]
temp['per_capita_area'] = raws['land_area_sqkm'].div(raws['persons_total'])

# merge temp and df on gisjoin and year. If any gisjoin is missing, drop that row
merged = pd.merge(temp, data, on=['gisjoin','year'], how='inner')

/var/folders/8h/w7q1fjwd5kn8xvqty5sp3h7r0000gn/T/ipykernel_91302/841716927.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['per_capita_area'] = raws['land_area_sqkm'].div(raws['persons_total'])


Next, we make a column `P(underage|C)` which is the probability of being underage given the county. This is computed as one minus the sum over all `sex_age_edus` columns (since these add up to all people aged 18+ in the county).

In [4]:
sex_age_edus = ['male_18_24_less_than_9th',
                'male_18_24_some_hs',
                'male_18_24_hs_grad',
                'male_18_24_some_college',
                'male_18_24_associates',
                'male_18_24_bachelors',
                'male_18_24_graduate',
                'male_25_34_less_than_9th',
                'male_25_34_some_hs',
                'male_25_34_hs_grad',
                'male_25_34_some_college',
                'male_25_34_associates',
                'male_25_34_bachelors',
                'male_25_34_graduate',
                'male_35_44_less_than_9th',
                'male_35_44_some_hs',
                'male_35_44_hs_grad',
                'male_35_44_some_college',
                'male_35_44_associates',
                'male_35_44_bachelors',
                'male_35_44_graduate',
                'male_45_64_less_than_9th',
                'male_45_64_some_hs',
                'male_45_64_hs_grad',
                'male_45_64_some_college',
                'male_45_64_associates',
                'male_45_64_bachelors',
                'male_45_64_graduate',
                'male_65plus_less_than_9th',
                'male_65plus_some_hs',
                'male_65plus_hs_grad',
                'male_65plus_some_college',
                'male_65plus_associates',
                'male_65plus_bachelors',
                'male_65plus_graduate',
                'female_18_24_less_than_9th',
                'female_18_24_some_hs',
                'female_18_24_hs_grad',
                'female_18_24_some_college',
                'female_18_24_associates',
                'female_18_24_bachelors',
                'female_18_24_graduate',
                'female_25_34_less_than_9th',
                'female_25_34_some_hs',
                'female_25_34_hs_grad',
                'female_25_34_some_college',
                'female_25_34_associates',
                'female_25_34_bachelors',
                'female_25_34_graduate',
                'female_35_44_less_than_9th',
                'female_35_44_some_hs',
                'female_35_44_hs_grad',
                'female_35_44_some_college',
                'female_35_44_associates',
                'female_35_44_bachelors',
                'female_35_44_graduate',
                'female_45_64_less_than_9th',
                'female_45_64_some_hs',
                'female_45_64_hs_grad',
                'female_45_64_some_college',
                'female_45_64_associates',
                'female_45_64_bachelors',
                'female_45_64_graduate',
                'female_65plus_less_than_9th',
                'female_65plus_some_hs',
                'female_65plus_hs_grad',
                'female_65plus_some_college',
                'female_65plus_associates',
                'female_65plus_bachelors',
                'female_65plus_graduate']
#make a list of P(x|C) columns for x in sex_age_edus
P_sae = [f'P({col}|C)' for col in sex_age_edus]

merged['P(underage|C)'] = 1 - merged[P_sae].sum(axis=1)
merged['P(non_voter|C)'] -= merged['P(underage|C)']

merged[merged.columns[-5:]].describe()

,P(democrat|C),P(other|C),P(republican|C),P(non_voter|C),P(underage|C)
count,12360.000000,12360.000000,12360.000000,12360.000000,12360.000000
mean,0.165146,0.012064,0.277635,0.317940,0.227216
std,0.079625,0.010955,0.085371,0.074824,0.034346
min,0.008703,0.000000,0.007386,0.026042,0.071341
25%,0.103428,0.005683,0.223412,0.266529,0.206919
50%,0.151652,0.008817,0.274800,0.315766,0.227229
75%,0.214947,0.014055,0.330726,0.366879,0.245678
max,0.540248,0.185841,0.729167,0.735494,0.406266


In [5]:
merged.to_csv('final_dataset.csv',index=False)

In [6]:
targets = ['P(democrat|C)', 'P(republican|C)', 'P(non_voter|C)', 'P(other|C)']

#divide each target by the sum of all targets
merged[targets] = merged[targets].div(merged[targets].sum(axis=1), axis=0)
merged[targets].describe()

,P(democrat|C),P(republican|C),P(non_voter|C),P(other|C)
count,12360.000000,12360.000000,12360.000000,12360.000000
mean,0.213071,0.359081,0.412284,0.015564
std,0.100790,0.108909,0.098661,0.014236
min,0.011791,0.009811,0.030581,0.000000
25%,0.134299,0.290189,0.344410,0.007398
50%,0.197019,0.356474,0.409682,0.011410
75%,0.276480,0.427924,0.476383,0.018036
max,0.688758,0.915152,0.975512,0.228487


In [7]:
list(merged.columns)

['year',
 'gisjoin',
 'median_household_income',
 'per_capita_income',
 'per_capita_area',
 'state',
 'county',
 'P(C)',
 'P(households_income_under_10k|C)',
 'P(households_income_10k_15k|C)',
 'P(households_income_15k_25k|C)',
 'P(households_income_25k_plus|C)',
 'P(persons_male|C)',
 'P(persons_female|C)',
 'P(male_never_married|C)',
 'P(male_married|C)',
 'P(male_separated|C)',
 'P(male_widowed|C)',
 'P(male_divorced|C)',
 'P(female_never_married|C)',
 'P(female_married|C)',
 'P(female_separated|C)',
 'P(female_widowed|C)',
 'P(female_divorced|C)',
 'P(male_18_24_less_than_9th|C)',
 'P(male_18_24_some_hs|C)',
 'P(male_18_24_hs_grad|C)',
 'P(male_18_24_some_college|C)',
 'P(male_18_24_associates|C)',
 'P(male_18_24_bachelors|C)',
 'P(male_18_24_graduate|C)',
 'P(male_25_34_less_than_9th|C)',
 'P(male_25_34_some_hs|C)',
 'P(male_25_34_hs_grad|C)',
 'P(male_25_34_some_college|C)',
 'P(male_25_34_associates|C)',
 'P(male_25_34_bachelors|C)',
 'P(male_25_34_graduate|C)',
 'P(male_35_44_l

In [ ]:
# start from scratch by reading in the final dataset
df = pd.read_csv('final_dataset.csv')

#list of election years
years = [2008,2012,2016,2020]

#dictinary of dataframes for each year
dfs={year: df[df['year']==year] for year in years}

# save each dataframe to a csv file
for year in years:
    dfs[year].to_csv(f'final_dataset_{year}.csv', index=False)

In [ ]:
#dictionary of data by election year (start from scratch)
dfs = {year: pd.read_csv(f'final_dataset_{year}.csv') for year in years}

#4 targets (along with P(underage|C), form a probability distribution)
targets = ['P(democrat|C)',
            'P(other|C)',
            'P(republican|C)',
            'P(non_voter|C)']

#dictionaries X, y, and county weights for each year
ys = {year: dfs[year][targets] for year in years}
wts = {year: dfs[year]['P(C)'] for year in years}
Xs = {year: dfs[year].drop(columns = targets + ['year','gisjoin','state','county']) for year in years}

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

standardized_Xs = {}
for i,y1 in enumerate(years):
    #standardize X for the year y1
    standardized_Xs[y1] = pd.DataFrame(scaler.fit_transform(Xs[y1]), columns = Xs[y1].columns)
    #save X, y and weights for year y1 to csv
    standardized_Xs[y1].to_csv(f'X_{y1}.csv', index = False)
    ys[y1].to_csv(f'y_{y1}.csv', index = False)
    wts[y1].to_csv(f'wts_{y1}.csv', index = False)

    # loop over years after y1
    for j in range(i+1,4):
        y2 = years[j]
        #concatentate Xs, ys, and wts for the two years along columns
        temp_X = pd.concat([Xs[y1], Xs[y2]])
        temp_y = pd.concat([ys[y1], ys[y2]])
        temp_wts = pd.concat([wts[y1], wts[y2]])

        #standardize the concatenated Xs
        temp_X = pd.DataFrame(scaler.fit_transform(temp_X), columns = temp_X.columns)

        #add standardized Xs, wts and ys to the corresponding dictionaries
        standardized_Xs[(y1,y2)] = temp_X
        wts[(y1,y2)] = temp_wts
        ys[(y1,y2)] = temp_y

        #save X, y and wts for the two years to csv
        temp_X.to_csv(f'X_{y1}_{y2}.csv', index = False)
        temp_y.to_csv(f'y_{y1}_{y2}.csv', index = False)
        temp_wts.to_csv(f'wts_{y1}_{y2}.csv', index = False)

        # loop over years after y2
        for k in range(j+1,4):
            y3 = years[k]
            #concatentate Xs, ys, and wts for the three years along columns
            temp_X = pd.concat([Xs[y1], Xs[y2], Xs[y3]])
            temp_y = pd.concat([ys[y1], ys[y2], ys[y3]])
            temp_wts = pd.concat([wts[y1], wts[y2], wts[y3]])

            #standardize the concatenated Xs
            temp_X = pd.DataFrame(scaler.fit_transform(temp_X), columns = temp_X.columns)
            #add wts and ys to the corresponding dictionaries
            standardized_Xs[(y1,y2,y3)] = temp_X
            wts[(y1,y2,y3)] = temp_wts
            ys[(y1,y2,y3)] = temp_y

            #save X, y and wts for the two years to csv
            temp_X.to_csv(f'X_{y1}_{y2}_{y3}.csv', index = False)
            temp_y.to_csv(f'y_{y1}_{y2}_{y3}.csv', index = False)
            temp_wts.to_csv(f'wts_{y1}_{y2}_{y3}.csv', index = False)
    